In [1]:
import json, copy
import operator
import datetime


In [2]:
from KTAL.RadarAnalysis.GridLassoTools import read_polys

time_keys = {'frame_time':'%Y-%m-%dT%H:%M:%S'}

# Read in some polygons
polys = read_polys('/data/VORTEX-SE/NALMA/realtime/VSE3sticknetA-AL-20160331-polygon.txt', 
                   sort_key = 'frame_time', time_keys=time_keys)


/Users/ebruning/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/ebruning/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
for p in polys:
    print p

{u'y_verts': [35.370535714285715, 35.450892857142854, 35.2734375, 34.941964285714285, 34.720982142857146, 35.018973214285715, 35.229910714285715, 35.370535714285715], u'x_verts': [-87.74516129032259, -88.03548387096775, -88.17741935483872, -88.1483870967742, -87.93870967741935, -87.73870967741937, -87.66774193548387, -87.74516129032259], u'title': u'Lasso Verts', 'created': datetime.datetime(2016, 4, 20, 22, 25, 57, 189000), u'frame_id': 1307, u'frame_time': datetime.datetime(2016, 3, 31, 21, 47), u'field': u'flash_extent', u'lon_verts': None, u'filename': u'/data/VORTEX-SE/NALMA/realtime/flashsort/results/grid_files/2016/Mar/31/NALMA_20160331_000000_86400_10src_0.0109deg-dx_flash_extent.nc', u'lat_verts': None}
{u'y_verts': [35.370535714285715, 35.450892857142854, 35.2734375, 34.941964285714285, 34.720982142857146, 35.018973214285715, 35.229910714285715, 35.370535714285715], u'x_verts': [-87.74516129032259, -88.03548387096775, -88.17741935483872, -88.1483870967742, -87.93870967741935,

In [4]:
# This prepares the polygon data for immedate use in the DC3 flash size statistics scripts
flash_stat_polys = tuple(zip(p['x_verts'], p['y_verts']) for p in polys)
flash_stat_left_time_edges = tuple(p['frame_time'] for p in polys)
dt = flash_stat_left_time_edges[1] - flash_stat_left_time_edges[0]
t_edge = flash_stat_left_time_edges + (flash_stat_left_time_edges[-1] + dt, )


In [5]:
# Set up a second, large-domain polygon with which to trim the polygons above.
filter_polys = [
    ( -88.2, 35.01),
    ( -87.5, 35.20),
    ( -87.0, 35.20),
    ( -87.0, 34.80),
    ( -88.2, 34.80),
    ( -88.2, 35.01),]


In [6]:
print polys[0]

{u'y_verts': [35.370535714285715, 35.450892857142854, 35.2734375, 34.941964285714285, 34.720982142857146, 35.018973214285715, 35.229910714285715, 35.370535714285715], u'x_verts': [-87.74516129032259, -88.03548387096775, -88.17741935483872, -88.1483870967742, -87.93870967741935, -87.73870967741937, -87.66774193548387, -87.74516129032259], u'title': u'Lasso Verts', 'created': datetime.datetime(2016, 4, 20, 22, 25, 57, 189000), u'frame_id': 1307, u'frame_time': datetime.datetime(2016, 3, 31, 21, 47), u'field': u'flash_extent', u'lon_verts': None, u'filename': u'/data/VORTEX-SE/NALMA/realtime/flashsort/results/grid_files/2016/Mar/31/NALMA_20160331_000000_86400_10src_0.0109deg-dx_flash_extent.nc', u'lat_verts': None}


In [7]:
from shapely.geometry import Polygon

filter_geom = Polygon(filter_polys)
trimmed_flash_stat_polys = [Polygon(poly).intersection(filter_geom) 
                            for poly in flash_stat_polys]

# sample_trimmed = trimmed_flash_stat_polys[0]
updated_coords = [{'x_verts':list(p.exterior.xy[0]), 
                  'y_verts':list(p.exterior.xy[1])} 
                 for p in trimmed_flash_stat_polys ]
assert len(polys) == len(updated_coords)
# old_polys = copy.deepcopy(polys)
for poly, new_coords in zip(polys, updated_coords):
    poly.update(new_coords)
    
print polys

[{u'y_verts': [34.8, 34.941964285714285, 35.02210403697055, 35.1468885137057, 35.018973214285715, 34.8, 34.8], u'x_verts': [-88.01368523949169, -88.1483870967742, -88.15540617958217, -87.69567389687376, -87.73870967741937, -87.88567596955419, -88.01368523949169], u'title': u'Lasso Verts', 'created': datetime.datetime(2016, 4, 20, 22, 25, 57, 189000), u'frame_id': 1307, u'frame_time': datetime.datetime(2016, 3, 31, 21, 47), u'field': u'flash_extent', u'lon_verts': None, u'filename': u'/data/VORTEX-SE/NALMA/realtime/flashsort/results/grid_files/2016/Mar/31/NALMA_20160331_000000_86400_10src_0.0109deg-dx_flash_extent.nc', u'lat_verts': None}, {u'y_verts': [34.8, 34.941964285714285, 35.02210403697055, 35.1468885137057, 35.018973214285715, 34.8, 34.8], u'x_verts': [-88.01368523949169, -88.1483870967742, -88.15540617958217, -87.69567389687376, -87.73870967741937, -87.88567596955419, -88.01368523949169], u'title': u'Lasso Verts', 'created': datetime.datetime(2016, 4, 20, 22, 25, 57, 189000), u

In [8]:
print polys[0]

{u'y_verts': [34.8, 34.941964285714285, 35.02210403697055, 35.1468885137057, 35.018973214285715, 34.8, 34.8], u'x_verts': [-88.01368523949169, -88.1483870967742, -88.15540617958217, -87.69567389687376, -87.73870967741937, -87.88567596955419, -88.01368523949169], u'title': u'Lasso Verts', 'created': datetime.datetime(2016, 4, 20, 22, 25, 57, 189000), u'frame_id': 1307, u'frame_time': datetime.datetime(2016, 3, 31, 21, 47), u'field': u'flash_extent', u'lon_verts': None, u'filename': u'/data/VORTEX-SE/NALMA/realtime/flashsort/results/grid_files/2016/Mar/31/NALMA_20160331_000000_86400_10src_0.0109deg-dx_flash_extent.nc', u'lat_verts': None}


## Further usage
The updated polygon dictionary above can be used anywhere read_polys had previously provided a dictionary of time-stamped polygons.